In [3]:
from google.colab import files
uploaded = files.upload()


Saving marketing_campaign_data_messy.csv to marketing_campaign_data_messy (1).csv


In [4]:
import pandas as pd
import numpy as np

# Load the messy data
df = pd.read_csv('marketing_campaign_data_messy.csv')

print(f"Loaded Dataset: {df.shape[0]} rows, {df.shape[1]} columns")

Loaded Dataset: 2020 rows, 12 columns


In [6]:
import pandas as pd

df = pd.read_csv("marketing_campaign_data_messy.csv")
df.head()


,Campaign_ID,Campaign_Name,Start_Date,End_Date,Channel,Impressions,Clicks,Spend,Conversions,Active,Clicks,Campaign_Tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24 00:00:00,2023-12-13,TikTok,16795,197,$102.82,20.0,Y,NaN,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06 00:00:00,2023-05-12,Facebook,1860,30,24.33,1.0,0,NaN,FA
2,CMP-00003,Q3_Winter_CMP-00003,2023-12-13 00:00:00,2023-12-20,Email,77820,843,1323.39,51.0,No,NaN,EM
3,CMP-00004,Q1_BlackFriday_CMP-00004,2023-10-30,2023-11-03,TikTok,55886,2019,2180.38,135.0,True,NaN,TI
4,CMP-00005,Q2_Winter_CMP-00005,2023-04-22 00:00:00,2023-04-23,Facebook,7265,169,252.44,30.0,Yes,NaN,FA


In [7]:
import pandas as pd
import numpy as np

# Load the messy data
df = pd.read_csv('marketing_campaign_data_messy.csv')

print(f"Loaded Dataset: {df.shape[0]} rows, {df.shape[1]} columns")

Loaded Dataset: 2020 rows, 12 columns


In [8]:
# ==========================================
# STEP 1: HEADER CLEANING
# ==========================================

print(df.columns.tolist())

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

print("FIX APPLIED")
print(df.columns.tolist())

[' Campaign_ID ', 'Campaign_Name', 'Start_Date', 'End_Date', 'Channel', 'Impressions', 'Clicks ', 'Spend', 'Conversions', 'Active', 'Clicks', 'Campaign_Tag']
FIX APPLIED
['campaign_id', 'campaign_name', 'start_date', 'end_date', 'channel', 'impressions', 'clicks', 'spend', 'conversions', 'active', 'clicks', 'campaign_tag']


In [9]:
# ==========================================
# STEP 2: TYPE CONVERSION & CURRENCY CLEANING
# ==========================================

dirty_spend_mask = df['spend'].astype(str).str.contains(r'\$')
print(df.loc[dirty_spend_mask,['campaign_id','spend']].head(3))

df['spend'] = df['spend'].astype(str).str.replace(r'[^\d.-]', '', regex=True)
df['spend'] = pd.to_numeric(df['spend'])

print("FIX APPLIED")
print(df.loc[dirty_spend_mask,['campaign_id','spend']].head(3))

   campaign_id     spend
0    CMP-00001   $102.82
21   CMP-00022   $2428.4
22   CMP-00023  $4726.22
FIX APPLIED
   campaign_id    spend
0    CMP-00001   102.82
21   CMP-00022  2428.40
22   CMP-00023  4726.22


In [10]:
# ==========================================
# STEP 3: CATEGORICAL TYPOS (FUZZY LOGIC)
# ==========================================

print(df['channel'].unique())

cleanup_map = {
    'Facebok': 'Facebook',
    'Insta_gram': 'Instagram',
    'Gogle': 'Google Ads',
    'Tik_Tok': 'TikTok',
    'E-mail': 'Email',
    'N/A': np.nan  # Handling the ghost value here too
}

df['channel'] = df['channel'].replace(cleanup_map)

print("FIX APPLIED")
print(df['channel'].unique())

['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' 'E-mail' nan 'Gogle'
 'Tik_Tok' 'Facebok' 'Insta_gram']
FIX APPLIED
['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' nan]


In [11]:
# ==========================================
# STEP 4: HANDLING MIXED BOOLEANS
# ==========================================

print(df['active'].unique())

bool_map = {

            'Yes': True,
            'Y': True,
            '1': True,
            1: True,
            'No': False,
            '0': False,
            0: False
}


df['active'] = df['active'].map(bool_map).fillna(False).astype(bool)

print("FIX APPLIED")
print(df['active'].unique())

['Y' '0' 'No' 'True' 'Yes' '1' 'False']
FIX APPLIED
[ True False]


/tmp/ipython-input-1915633584.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['active'] = df['active'].map(bool_map).fillna(False).astype(bool)


In [12]:
# ==========================================
# STEP 5: DATE PARSING
# ==========================================

print(df['start_date'].dtype)

df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
df['end_date'] = pd.to_datetime(df['end_date'], dayfirst=True, errors='coerce')

print("FIX APPLIED")
print(df['start_date'].dtype)


object
FIX APPLIED
datetime64[ns]


/tmp/ipython-input-877702922.py:8: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['end_date'] = pd.to_datetime(df['end_date'], dayfirst=True, errors='coerce')


In [13]:
df = df.loc[:, ~df.columns.duplicated()]

In [14]:
# ==========================================
# STEP 6: LOGICAL INTEGRITY (CLICKS vs IMPRESSIONS)
# ==========================================

impossible_mask = df['clicks'] > df['impressions']
print(df.loc[impossible_mask,['campaign_id','impressions','clicks']].head(3))

Empty DataFrame
Columns: [campaign_id, impressions, clicks]
Index: []


In [15]:
# ==========================================
# STEP 7: LOGICAL INTEGRITY (TIME TRAVEL)
# ==========================================

time_travel_mask = df['end_date'] < df['start_date']
print(df.loc[time_travel_mask,['campaign_id','start_date','end_date']].head(3))

df.loc[time_travel_mask, 'end_date'] = df.loc[time_travel_mask, 'start_date'] + pd.Timedelta(days=30)

print("FIX APPLIED")
print(df.loc[time_travel_mask,['campaign_id','start_date','end_date']].head(3))

   campaign_id start_date   end_date
23   CMP-00024 2023-05-06 2023-05-01
54   CMP-00055 2023-09-01 2023-08-27
71   CMP-00072 2023-02-01 2023-01-27
FIX APPLIED
   campaign_id start_date   end_date
23   CMP-00024 2023-05-06 2023-06-05
54   CMP-00055 2023-09-01 2023-10-01
71   CMP-00072 2023-02-01 2023-03-03


In [16]:
# ==========================================
# STEP 8: HANDLING OUTLIERS (WINSORIZING)
# ==========================================

Q1 = df['spend'].quantile(0.25)
Q3 = df['spend'].quantile(0.75)

IQR = Q3 - Q1
upper_limit = Q3 + (3 * IQR)

outlier_mask = df['spend'] > upper_limit
print(df.loc[outlier_mask,['campaign_id','spend']].head(3))

print("FIX APPLIED")
df.loc[outlier_mask, 'spend'] = upper_limit
print(df.loc[outlier_mask,['campaign_id','spend']].head(3))

     campaign_id      spend
789    CMP-00790  500000.00
1443   CMP-01444    8921.51
1460   CMP-01461  500000.00
FIX APPLIED
     campaign_id      spend
789    CMP-00790  8603.5375
1443   CMP-01444  8603.5375
1460   CMP-01461  8603.5375


In [17]:
# ==========================================
# STEP 9: STRING PARSING (FEATURE EXTRACTION)
# ==========================================

print(df['campaign_name'].head(3))

df['season'] = df['campaign_name'].str.extract(r'Q\d_([^_]+)_')

print("FIX APPLIED")

print(df[['campaign_name','season']].head(3))

0    Q4_Summer_CMP-00001
1    Q1_Launch_CMP-00002
2    Q3_Winter_CMP-00003
Name: campaign_name, dtype: object
FIX APPLIED
         campaign_name  season
0  Q4_Summer_CMP-00001  Summer
1  Q1_Launch_CMP-00002  Launch
2  Q3_Winter_CMP-00003  Winter


In [18]:
df.to_csv("marketing_companies_cleaned.csv", index=False)


In [19]:
import os
os.listdir()


['.config',
 'marketing_campaign_data_messy (1).csv',
 'marketing_companies_cleaned.csv',
 'marketing_campaign_data_messy.csv',
 'sample_data']

In [20]:
from google.colab import files
files.download("marketing_companies_cleaned.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>